In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import scipy.signal as signalpy
import wave
import sys
import math
import contextlib
import IPython
import os
import librosa
import librosa.display
import pandas as pd
import sklearn.datasets as datasets
import shutil
import scipy
from scipy.interpolate import interp1d
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.utils import to_categorical


### Signals as arrays

### Loading pandas dataframe from .npy file saved previously (whole signal in one array)

In [2]:
# Loading dataframe with signals saved as arrays. Looks like it does not work with KNN

pcg_dataframe = pd.DataFrame(data = np.load('arrays/pcg_dataframe_binarized_padded.npy' , allow_pickle=True) , columns = [
                                'Patient ID', 'Age', 'Sex' , 'Height' , 'Weight' , 'Pregnancy status' , 'Murmur', 'Murmur locations',
                                 'Most audible location', 'count_auscultation_locations' , 'current_auscultation_location', 'signal'])
display(pcg_dataframe)

,Patient ID,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,count_auscultation_locations,current_auscultation_location,signal
0,2530,2.0,1,98.0,15.9,0,0,NaN,-1.0,4,TV,"[0.044283557683229446, 0.08357305079698563, 0...."
1,2530,2.0,1,98.0,15.9,0,0,NaN,-1.0,4,PV,"[0.03783801943063736, 0.06029642000794411, 0.0..."
2,2530,2.0,1,98.0,15.9,0,0,NaN,-1.0,4,MV,"[0.18855691, 0.04248511, -0.017912444, 0.01127..."
3,2530,2.0,1,98.0,15.9,0,0,NaN,-1.0,4,AV,"[0.006233137, -0.0059207836, -0.016664337, 0.0..."
4,9979,2.0,1,103.0,13.1,0,1,AV+MV+PV+TV,1.0,4,TV,"[-0.10014393, -0.82379806, -0.6746199, -0.4551..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3002,85345,2.0,1,132.0,38.1,0,0,NaN,-1.0,2,PV,"[0.00654612248763442, 0.017596282064914703, 0...."
3003,85345,2.0,1,132.0,38.1,0,0,NaN,-1.0,2,AV,"[0.03042028099298477, 0.00832417979836464, -0...."
3004,85349,-1.0,1,-1.0,-1.0,1,0,NaN,-1.0,3,AV,"[0.0020578710827976465, -0.008989883586764336,..."
3005,85349,-1.0,1,-1.0,-1.0,1,0,NaN,-1.0,3,PV,"[0.0742131769657135, 0.05925178900361061, 0.00..."


In [3]:
if 'Murmur locations' in pcg_dataframe.columns:
    pcg_dataframe = pcg_dataframe.drop(columns=['Patient ID' ,'Murmur locations' , 'current_auscultation_location'])

display(pcg_dataframe)

print(pcg_dataframe['signal'][0].shape)

,Age,Sex,Height,Weight,Pregnancy status,Murmur,Most audible location,count_auscultation_locations,signal
0,2.0,1,98.0,15.9,0,0,-1.0,4,"[0.044283557683229446, 0.08357305079698563, 0...."
1,2.0,1,98.0,15.9,0,0,-1.0,4,"[0.03783801943063736, 0.06029642000794411, 0.0..."
2,2.0,1,98.0,15.9,0,0,-1.0,4,"[0.18855691, 0.04248511, -0.017912444, 0.01127..."
3,2.0,1,98.0,15.9,0,0,-1.0,4,"[0.006233137, -0.0059207836, -0.016664337, 0.0..."
4,2.0,1,103.0,13.1,0,1,1.0,4,"[-0.10014393, -0.82379806, -0.6746199, -0.4551..."
...,...,...,...,...,...,...,...,...,...
3002,2.0,1,132.0,38.1,0,0,-1.0,2,"[0.00654612248763442, 0.017596282064914703, 0...."
3003,2.0,1,132.0,38.1,0,0,-1.0,2,"[0.03042028099298477, 0.00832417979836464, -0...."
3004,-1.0,1,-1.0,-1.0,1,0,-1.0,3,"[0.0020578710827976465, -0.008989883586764336,..."
3005,-1.0,1,-1.0,-1.0,1,0,-1.0,3,"[0.0742131769657135, 0.05925178900361061, 0.00..."


(27473,)


In [4]:
y_together = pcg_dataframe['Murmur']

x_together = pcg_dataframe
x_together = x_together.drop(columns=['Murmur'])

display(x_together)
display(y_together)

,Age,Sex,Height,Weight,Pregnancy status,Most audible location,count_auscultation_locations,signal
0,2.0,1,98.0,15.9,0,-1.0,4,"[0.044283557683229446, 0.08357305079698563, 0...."
1,2.0,1,98.0,15.9,0,-1.0,4,"[0.03783801943063736, 0.06029642000794411, 0.0..."
2,2.0,1,98.0,15.9,0,-1.0,4,"[0.18855691, 0.04248511, -0.017912444, 0.01127..."
3,2.0,1,98.0,15.9,0,-1.0,4,"[0.006233137, -0.0059207836, -0.016664337, 0.0..."
4,2.0,1,103.0,13.1,0,1.0,4,"[-0.10014393, -0.82379806, -0.6746199, -0.4551..."
...,...,...,...,...,...,...,...,...
3002,2.0,1,132.0,38.1,0,-1.0,2,"[0.00654612248763442, 0.017596282064914703, 0...."
3003,2.0,1,132.0,38.1,0,-1.0,2,"[0.03042028099298477, 0.00832417979836464, -0...."
3004,-1.0,1,-1.0,-1.0,1,-1.0,3,"[0.0020578710827976465, -0.008989883586764336,..."
3005,-1.0,1,-1.0,-1.0,1,-1.0,3,"[0.0742131769657135, 0.05925178900361061, 0.00..."


0       0
1       0
2       0
3       0
4       1
       ..
3002    0
3003    0
3004    0
3005    0
3006    0
Name: Murmur, Length: 3007, dtype: object

In [5]:
labels = y_together.to_numpy()
signals = x_together.to_numpy()
print(signals)

[[2.0 1 98.0 ... -1.0 4
  array([ 0.04428356,  0.08357305,  0.08483126, ..., -0.02213163,
          0.0360193 ,  0.        ])                              ]
 [2.0 1 98.0 ... -1.0 4
  array([ 0.03783802,  0.06029642,  0.05549384, ..., -0.06139453,
         -0.03247141,  0.        ])                              ]
 [2.0 1 98.0 ... -1.0 4
  array([ 0.18855691,  0.04248511, -0.01791244, ...,  0.28258502,
          0.23062131,  0.14469288], dtype=float32)               ]
 ...
 [-1.0 1 -1.0 ... -1.0 3
  array([ 0.00205787, -0.00898988,  0.00249804, ...,  0.00588843,
          0.19711504, -0.05632662])                              ]
 [-1.0 1 -1.0 ... -1.0 3
  array([ 0.07421318,  0.05925179,  0.00589764, ..., -0.02336963,
          0.00817627,  0.        ])                              ]
 [-1.0 1 -1.0 ... -1.0 3
  array([ 0.08635268,  0.01259144, -0.02330904, ...,  0.01838567,
         -0.01804134,  0.03651306])                              ]]


### Spliting the data into train and test set. 
We need to stratify the data based on the label (Murmur - present/absent) to ensure that the train and test set have the same proportion of the label.

Q: Why are we doing that?

A: Our data is imbalanced. Only ~20% of the data has Murmur present. We don't want our model to be biased, so we need to stratify the data.

In [7]:
x_together_test, x_together_train, y_together_test, y_together_train = train_test_split(signals, labels, test_size=0.25, stratify=y_together)
# x_together_test, x_together_train, x_together_test, y_together_train = train_test_split(x_together, y_together, test_size=0.25, stratify=y_together)

### K Nearest Neighbors

In [8]:
classifier = KNeighborsClassifier()

classifier.fit(x_together_train, y_together_train)

train_accuracy = classifier.score(x_together_train, y_together_train)
test_accuracy = classifier.score(x_together_test, y_together_train)

ValueError: setting an array element with a sequence.